In [1]:
%%bigquery df
SELECT *
FROM `stockswhatsup.stocks_data_historical.stocks_data_all`

Query is running:   0%|          |

Downloading:   0%|          |

In [2]:
!pip install pmdarima

  Using cached pmdarima-2.0.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl.metadata (7.8 kB)
Using cached pmdarima-2.0.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl (2.1 MB)


In [3]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression
from pmdarima.arima import auto_arima
from pylab import rcParams
from google.cloud import bigquery

def process_stock_data(stock_df, stock_name):
    start_date = stock_df.date.min()
    end_date = stock_df.date.max()

    complete_date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    complete_date_df = pd.DataFrame({f'{stock_name}_complete_date': complete_date_range})

    complete_date_df[f'{stock_name}_complete_date'] = pd.to_datetime(complete_date_df[f'{stock_name}_complete_date'])
    stock_df['date'] = pd.to_datetime(stock_df['date'])
    complete_date_df = complete_date_df.reset_index(drop=True)
    stock_df = stock_df.reset_index(drop=True)
    continuous_df = pd.merge(complete_date_df, stock_df, how='left', left_on=f'{stock_name}_complete_date', right_on='date')

    continuous_df.set_index(f'{stock_name}_complete_date', inplace=True)
    continuous_df.index = pd.DatetimeIndex(continuous_df.index).to_period('D')

    continuous_df = continuous_df[['volume_weighted_avg_price']].interpolate(method='time')

    # Convert the DatetimeIndex to Timestamp
    continuous_df.index = continuous_df.index.to_timestamp()

    # ADF Test
    adft_result = adfuller(continuous_df['volume_weighted_avg_price'], autolag='AIC')

    if adft_result[1] < 0.05:
        continuous_df = np.log(continuous_df)

    train_data, test_data = continuous_df[:int(len(continuous_df) * 0.9)], continuous_df[int(len(continuous_df) * 0.9):]

    return train_data, test_data

def build_arima_model(train_data):
    model_autoARIMA = auto_arima(train_data, start_p=0, start_q=0,
                                test='adf', max_p=3, max_q=3, m=1,
                                d=None, seasonal=False, start_P=0,
                                D=0, trace=True, error_action='ignore',
                                suppress_warnings=True, stepwise=True)

    model = ARIMA(train_data, order=model_autoARIMA.order)
    fitted = model.fit()

    return fitted.forecast(638, alpha=0.05)

def build_forecast_df(stock_fc, stock_name):
    fc_df = pd.DataFrame(stock_fc)
    fc_df['stock_symbol'] = stock_name.upper()
    fc_df.reset_index(inplace=True) 
    return fc_df

stocks = ['META', 'AAPL', 'AMZN', 'GOOGL', 'NFLX']

forecasted_dfs = []

for stock in stocks:
    stock_df = df[df['stock_symbol'] == stock.upper()][['volume_weighted_avg_price', 'date']]
    train_data, test_data = process_stock_data(stock_df, stock)
    stock_fc = build_arima_model(train_data)
    stock_fc_df = build_forecast_df(stock_fc, stock)
    forecasted_dfs.append(stock_fc_df)

forecasted_df_combined = pd.concat(forecasted_dfs, ignore_index=True)

client = bigquery.Client()

forecast_table_id = "stockswhatsup.stocks_data_historical.stocks_forecast"
forecasted_df_combined.rename(columns={'index':'date'}, inplace=True)

forecast_table_config = bigquery.LoadJobConfig(
      schema=[
          bigquery.SchemaField("date", bigquery.enums.SqlTypeNames.DATE),
          bigquery.SchemaField("predicted_mean", bigquery.enums.SqlTypeNames.FLOAT),
          bigquery.SchemaField("stock_symbol", bigquery.enums.SqlTypeNames.STRING),
      ],
      write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
  )

try:
    forecast_table_job = client.load_table_from_dataframe(
        forecasted_df_combined, forecast_table_id, job_config=forecast_table_config
    )
    print('Success: Uploaded Forecasted data to BigQuery')
except Exception as e:
    print(f'Failure: {e}')

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3435.524, Time=0.06 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3396.254, Time=0.10 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3396.382, Time=0.08 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3446.954, Time=0.03 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=3397.901, Time=0.09 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3397.801, Time=0.13 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=3399.561, Time=0.31 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=3402.313, Time=0.04 sec

Best model:  ARIMA(1,1,0)(0,0,0)[0] intercept
Total fit time: 0.859 seconds
Performing stepwise search to minimize aic


/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=2742.062, Time=0.06 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=2721.440, Time=0.10 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=2721.633, Time=0.09 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=2740.065, Time=0.03 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=2723.383, Time=0.11 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2723.370, Time=0.16 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=2725.167, Time=0.35 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=2719.443, Time=0.04 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=2721.385, Time=0.18 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=2721.372, Time=0.07 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=2719.636, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=2723.169, Time=0.18 sec

Best model:  ARIMA(1,1,0)(0,0,0)[0]          
Total fit time: 1.445 seconds
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=2906.180, Time=0.06 sec


/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=2886.354, Time=0.10 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=2888.151, Time=0.08 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=2904.692, Time=0.02 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=2887.742, Time=0.09 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2887.766, Time=0.15 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=2889.742, Time=0.14 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=2884.705, Time=0.04 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=2886.070, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=2886.092, Time=0.07 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=2886.537, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=2888.070, Time=0.07 sec

Best model:  ARIMA(1,1,0)(0,0,0)[0]          
Total fit time: 0.929 seconds


...

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=2532.497, Time=0.06 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=2505.533, Time=0.10 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=2505.598, Time=0.10 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=2530.524, Time=0.03 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=2507.147, Time=0.09 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2507.192, Time=0.17 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=2507.223, Time=0.46 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=2503.548, Time=0.03 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=2505.163, Time=0.04 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=2505.208, Time=0.07 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=2503.616, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=2505.256, Time=0.15 sec

Best model:  ARIMA(1,1,0)(0,0,0)[0]          
Total fit time: 1.369 seconds
Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=954

/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


 ARIMA(1,0,0)(0,0,0)[0]             : AIC=inf, Time=0.12 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=inf, Time=0.11 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=4478.683, Time=0.12 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=4478.379, Time=0.17 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=inf, Time=0.12 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=4480.014, Time=0.85 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=4479.402, Time=0.78 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=4478.784, Time=0.16 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=inf, Time=0.18 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=4482.218, Time=0.21 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=4476.948, Time=0.41 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=4477.373, Time=0.25 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=inf, Time=0.26 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=4478.555, Time=1.44 sec
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=4477.747, Time=2.15 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept  

/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


Success: Uploaded Forecasted data to BigQuery
